In [2]:
import pandas as pd
from textblob import TextBlob
import nltk
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Flatten,SimpleRNN
from keras import regularizers
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

In [2]:
c
len(df)

232074

In [3]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,1,Am I weird I don t get affected by compliments...,non-suicide
2,2,Finally is almost over So I can never hear...,non-suicide
3,3,i need helpjust help me im crying so hard,suicide
4,4,I m so lostHello my name is Adam and I ve b...,suicide
5,5,Honetly idkI dont know what im even doing here...,suicide
6,6,Trigger warning Excuse for self inflicted bu...,suicide
7,7,It ends tonight I can t do it anymore I quit,suicide
8,8,Everyone wants to be edgy and it s making me...,non-suicide
9,9,My life is over at years oldHello all I am a...,suicide


In [6]:
l=[]
for i in range(len(df)):
    l.append(TextBlob(df.iloc[i,1]).sentiment.polarity)
l1=[]
for i in l:
    if i>=0.2:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[:,2].tolist()


In [7]:
from sklearn.metrics import accuracy_score
print(accuracy_score(l1,l2))

0.5741358359833502


In [89]:
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\metul\AppData\Roaming\nltk_data...


True

In [8]:
df=df.iloc[0:100,:]
len(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   text        100 non-null    object
 2   class       100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [11]:

def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df
#Generating Embeddings using tokenizer
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)
#Model Building
model = Sequential()
'''
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))'''
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(150, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1568, 120)         60000     
                                                                 
 spatial_dropout1d_8 (Spatia  (None, 1568, 120)        0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 300)               505200    
                                                                 
 dense_16 (Dense)            (None, 150)               45150     
                                                                 
 dense_17 (Dense)            (None, 3)                 453       
                                                                 
Total params: 610,803
Trainable params: 610,803
Non-trainable params: 0
________________________________________________

[0.6433237195014954, 0.6000000238418579]

In [101]:
print(X_train,len(X_train))
print(X_test,len(X_test))
print(y_train,len(y_train))
print(y_test,len(y_test))

[[  0   0   0 ...  20   7   3]
 [  0   0   0 ...  73 229 113]
 [  0   0   0 ...   2  15   6]
 ...
 [  0   0   0 ...  83  10  11]
 [  0   0   0 ... 258   2  66]
 [  0   0   0 ...  54  58  33]] 80
[[  0   0   0 ...  21   5 282]
 [  0   0   0 ...   5   8   6]
 [  0   0   0 ...  28  48   7]
 ...
 [  0   0   0 ...  52   2 333]
 [  0   0   0 ...  26   9  21]
 [  0   0   0 ...   3   7  81]] 20
49    non-suicide
25        suicide
48        suicide
86        suicide
15    non-suicide
         ...     
87        suicide
66    non-suicide
19        suicide
29    non-suicide
9         suicide
Name: class, Length: 80, dtype: object 80
99        suicide
85        suicide
93        suicide
92    non-suicide
5         suicide
62        suicide
72    non-suicide
80    non-suicide
55    non-suicide
39    non-suicide
73        suicide
35    non-suicide
24    non-suicide
23    non-suicide
56    non-suicide
68    non-suicide
20        suicide
16        suicide
52    non-suicide
63    non-suicide
Name: clas

In [103]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
l1=sentiment_pipeline(df.iloc[0:100].tolist())


l1=[]
for i in l:
    if i>=0:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[0:100,2].tolist()
print(accuracy_score(l1,l2))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

d:\tools2\anaconda\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\metul\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

AttributeError: 'DataFrame' object has no attribute 'tolist'

SECOND IMPLEMENTATION


In [9]:
df = pd.read_csv('/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv',
                 lineterminator='\n')

In [8]:
df=pd.read_csv("/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed2.csv")
#df = pd.read_csv( "/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv", index_col=None, header=0, engine='python' )
df.head()

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [10]:
def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df
#Generating Embeddings using tokenizer
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)

In [4]:
#Model Building
model = Sequential()
'''
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))'''
'''
model.add(Embedding(50, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model.add(MaxPooling1D(5))
model.add(Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model.add(GlobalMaxPooling1D())
model.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(15, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])'''

model.add(Embedding(50, 100, input_length=X.shape[1]))
#model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(SimpleRNN(10, return_sequences=True))
model.add(LSTM(10, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(15, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9685, 100)         5000      
                                                                 
 lstm (LSTM)                 (None, 10)                4440      
                                                                 
 dense (Dense)               (None, 15)                165       
                                                                 
 dense_1 (Dense)             (None, 3)                 48        
                                                                 
Total params: 9,653
Trainable params: 9,653
Non-trainable params: 0
_________________________________________________________________
None
[[  0   0   0 ...   1  13   3]
 [  0   0   0 ...  37   7  57]
 [  0   0   0 ...  19   1  47]
 ...
 [  0   0   0 ...  52   1 260]
 [  0   0   0 ...  21 104  75]
 [  0   0   0 ...  49

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "d:\tools2\anaconda\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\tools2\anaconda\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\tools2\anaconda\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\tools2\anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "d:\tools2\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\tools2\anaconda\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "d:\tools2\anaconda\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    File "d:\tools2\anaconda\